In [3]:
%pip install supervision
%pip -q install roboflow
%pip -q install ultralytics==8.0.28


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.0/69.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.

In [4]:
from google.colab import drive
drive.mount('/content/drive')
dataset_location = "/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3"

Mounted at /content/drive


In [5]:
import supervision as sv



In [6]:
from ultralytics import YOLO
from PIL import Image
def predict_img(img_path, model_path):

  model = YOLO(model_path)
  source = img_path

  results = model(source)
  for r in results:
    try:
      return r.boxes.xywhn.cpu().numpy()[0]
    except Exception as e:
      return [0,0,0,0]

In [1]:
def calcula_iou_numpy(predict, label):
    # caixa1 e caixa2 devem ser arrays NumPy no formato [x, y, largura, altura]
    x1, y1, w1, h1 = predict
    try:
      id2, x2, y2, w2, h2 = label
    except Exception as e:
      id2, x2, y2, w2, h2 = label[0]

    # Calcula as coordenadas dos cantos da interseção
    x_intersecao = max(x1, x2)
    y_intersecao = max(y1, y2)
    largura_intersecao = min(x1 + w1, x2 + w2) - x_intersecao
    altura_intersecao = min(y1 + h1, y2 + h2) - y_intersecao

    # Verifica se há interseção
    if largura_intersecao <= 0 or altura_intersecao <= 0:
        return 0.0

    # Calcula a área da interseção e da união
    area_intersecao = largura_intersecao * altura_intersecao
    area_uniao = w1 * h1 + w2 * h2 - area_intersecao

    # Calcula o IoU
    iou = area_intersecao / area_uniao

    return iou

In [8]:
import os
import numpy as np
source = ['/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/test','/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/train','/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/valid']
yolo_s = []
yolo_m = []
yolo_n = []
yolo_l = []
yolo_x = []


for filepath in source:
  for filename in os.listdir(f"{filepath}/labels"):
      labels = np.loadtxt(os.path.join(f"{filepath}/labels", filename))
      images = os.path.join(f"{filepath}/images", filename).split('.txt')[0] + '.jpg'
      print(images)

      predict_s = predict_img(images,'/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/trainyolov8s/weights/best.pt')
      print(predict_s)
      yolo_s.append(calcula_iou_numpy(predict_s, labels))
      with open(f"/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/trainyolov8s/inferencias/{filename}", 'a') as file:
       file.write(str(predict_s))

      predict_m = predict_img(images,'/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/train2yolov8m/weights/best.pt')
      yolo_m.append(calcula_iou_numpy(predict_m, labels))
      with open(f"/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/train2yolov8m/inferencias/{filename}", 'a') as file:
        file.write(str(predict_m))

      predict_n = predict_img(images,'/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/train5yolov8n/weights/best.pt')
      yolo_n.append(calcula_iou_numpy(predict_n, labels))
      with open(f"/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/train5yolov8n/inferencias/{filename}", 'a') as file:
        file.write(str(predict_n))

      predict_l = predict_img(images,'/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/train3yolov8L/weights/best.pt')
      yolo_l.append(calcula_iou_numpy(predict_l, labels))
      with open(f"/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/train3yolov8L/inferencias/{filename}", 'a') as file:
        file.write(str(predict_l))

      predict_x = predict_img(images,'/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/train4yolov8X/weights/best.pt')
      yolo_x.append(calcula_iou_numpy(predict_x, labels))
      with open(f"/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/train4yolov8X/inferencias/{filename}", 'a') as file:
        file.write(str(predict_x))









with open(f"/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/train4yolov8X/iou.txt", 'w') as file:
        file.write(str(yolo_x))

with open(f"/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/train2yolov8m/iou.txt", 'w') as file:
        file.write(str(yolo_m))

with open(f"/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/train3yolov8L/iou.txt", 'w') as file:
        file.write(str(yolo_l))

with open(f"/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/train5yolov8n/iou.txt", 'w') as file:
        file.write(str(yolo_n))

with open(f"/content/drive/MyDrive/5Modelos_Yolov8_DatasetTumor/trainyolov8s/iou.txt", 'w') as file:
        file.write(str(yolo_s))


[    0.68973     0.34598     0.27232     0.26339]


Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs

image 1/1 /content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/train/images/73_jpg.rf.1f7308362b30913ff9b93168cdb7e331.jpg: 640x640 1 Tumor, 25.0ms
Speed: 0.5ms pre-process, 25.0ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
Ultralytics YOLOv8.0.28 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

image 1/1 /content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/train/images/73_jpg.rf.1f7308362b30913ff9b93168cdb7e331.jpg: 640x640 1 Tumor, 16.9ms
Speed: 0.6ms pre-process, 16.9ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
Ultralytics YOLOv8.0.28 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 43607379 parameters, 0 gradients, 164.8 GFLOPs

image 1/1 /content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/train/images/73_jpg.rf.1f7

/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/train/images/Te-me_0115_jpg.rf.7306db07d1940310c3f71354a47c05e6.jpg


Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs

image 1/1 /content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/train/images/Te-me_0115_jpg.rf.7306db07d1940310c3f71354a47c05e6.jpg: 640x640 16.6ms
Speed: 0.5ms pre-process, 16.6ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)
Ultralytics YOLOv8.0.28 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)


[0, 0, 0, 0]


...

Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs

image 1/1 /content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/valid/images/1_jpg.rf.4aa6665f5fb204ef7135ac63df27e348.jpg: 640x640 1 Tumor, 23.0ms
Speed: 0.5ms pre-process, 23.0ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
Ultralytics YOLOv8.0.28 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

image 1/1 /content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/valid/images/1_jpg.rf.4aa6665f5fb204ef7135ac63df27e348.jpg: 640x640 1 Tumor, 16.8ms
Speed: 0.5ms pre-process, 16.8ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
Ultralytics YOLOv8.0.28 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 43607379 parameters, 0 gradients, 164.8 GFLOPs

image 1/1 /content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/valid/images/1_jpg.rf.4aa666

/content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/valid/images/Te-me_0144_jpg.rf.fda5a4dea539b5318243c348ccdaef5f.jpg


Model summary (fused): 168 layers, 11125971 parameters, 0 gradients, 28.4 GFLOPs

image 1/1 /content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/valid/images/Te-me_0144_jpg.rf.fda5a4dea539b5318243c348ccdaef5f.jpg: 640x640 1 Tumor, 16.0ms
Speed: 0.5ms pre-process, 16.0ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
Ultralytics YOLOv8.0.28 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)


[    0.52148     0.56836     0.12109     0.14062]


Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs

image 1/1 /content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/valid/images/Te-me_0144_jpg.rf.fda5a4dea539b5318243c348ccdaef5f.jpg: 640x640 1 Tumor, 20.5ms
Speed: 0.5ms pre-process, 20.5ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)
Ultralytics YOLOv8.0.28 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3005843 parameters, 0 gradients, 8.1 GFLOPs

image 1/1 /content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/valid/images/Te-me_0144_jpg.rf.fda5a4dea539b5318243c348ccdaef5f.jpg: 640x640 1 Tumor, 18.9ms
Speed: 0.5ms pre-process, 18.9ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 640)
Ultralytics YOLOv8.0.28 🚀 Python-3.10.12 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 268 layers, 43607379 parameters, 0 gradients, 164.8 GFLOPs

image 1/1 /content/drive/MyDrive/CsiLab-BrainTumor-Detection-3/valid/imag

In [9]:
from statistics import mean

print('Media de iou com yolov8s: ',mean(yolo_s))
print('Media de iou com yolov8m: ',mean(yolo_m))
print('Media de iou com yolov8n: ',mean(yolo_n))
print('Media de iou com yolov8l: ',mean(yolo_l))
print('Media de iou com yolov8x: ',mean(yolo_x))

Media de iou com yolov8s:  0.6782114523583231
Media de iou com yolov8m:  0.7743599869950156
Media de iou com yolov8n:  0.7907445274511441
Media de iou com yolov8l:  0.7747715247510605
Media de iou com yolov8x:  0.7811062242386428
